## Plotting accelerometer data vs mount motions

Craig Lage - Apr 7, 2022

In [ ]:
import sys, time, datetime
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle as pkl
from astropy.time import Time, TimeDelta
import astropy.units as u
from lsst.daf.butler import Butler
from lsst_efd_client import EfdClient

In [ ]:
# Get EFD client and the butler
client = EfdClient('summit_efd')

In [ ]:
filename = '20220406T223707Z.pkl'
# Unpickle the accel dataframe
file = open(f'/scratch/labJackData/{filename}', 'rb')
df = pkl.load(file)
file.close()

In [ ]:
timestamp = filename.split('.')[0]
# Mask out bad regions
df = df.where(abs(df['ELM2']) < 10.0) 
print(f"Approximately {len(df) / 200:.2f} seconds of data")
# Select a subset of the data if desired
# and apply a rolling average
rolling = 100 # number of data points to average
timeSubset = True
tstart = "2022-04-07 01:14:30"
tend = "2022-04-07 01:15:30"
if timeSubset:
    subdf = df.loc[tstart:tend]
    subdf = subdf.rolling(rolling).sum() / rolling
print(f"Approximately {len(subdf) / 200:.2f} seconds of data")

tstart = Time(tstart)
tend = Time(tend)
az = await client.select_packed_time_series("lsst.sal.ATMCS.mount_AzEl_Encoders", \
                                            "azimuthCalculatedAngle",  tstart, tend)
el = await client.select_packed_time_series("lsst.sal.ATMCS.mount_AzEl_Encoders", \
                                            "elevationCalculatedAngle",  tstart, tend)
doOffset = True
if doOffset:
    offset = (tstart.jd - az.index[0].to_julian_date()) * 86400.0
    az.index += pd.DateOffset(seconds=offset)
    el.index += pd.DateOffset(seconds=offset)
# Calculate the tracking errors
az_vals = np.array(az.values.tolist())[:,0]
el_vals = np.array(el.values.tolist())[:,0]
times = np.array(az.values.tolist())[:,1]
times = times - times [0]

# Fit with a quadratic
az_fit = np.polyfit(times, az_vals, 4)
el_fit = np.polyfit(times, el_vals, 4)

az_model = az_fit[0] * times * times * times * times + az_fit[1] * times * times * times \
    + az_fit[2] * times *times + az_fit[3] * times + az_fit[4]
el_model = el_fit[0] * times * times * times * times + el_fit[1] * times * times * times \
    + el_fit[2] * times * times + el_fit[3] * times + el_fit[4]

# Errors in arcseconds
az_error = (az_vals - az_model) * 3600
el_error = (el_vals - el_model) * 3600


fig = plt.figure(figsize = (10,8))
plt.subplots_adjust(hspace=1.0, wspace=1.5)

plt.suptitle(f"Mount Tracking vs Accel 20220406", fontsize = 18)
plt.subplots_adjust(wspace=0.5, hspace=0.5)
plt.subplot(2,2,1)
name = "AZM2"
plt.title(name, fontsize=12)
subdf[name].plot(color='red')
plt.ylabel('Acceleration(g)')
plt.subplot(2,2,2)
name = "AZT"
plt.title(name, fontsize=12)
subdf[name].plot(color='red')
plt.ylabel('Acceleration(g)')
plt.subplot(2,2,3)
name = "Az"
plt.title(name, fontsize=12)
#plt.plot(times, az_vals)
az['azimuthCalculatedAngle'].plot()
plt.ylabel('Degrees')
plt.subplot(2,2,4)
name = "El"
plt.title(name, fontsize=12)
#plt.plot(times, el_vals)
el['elevationCalculatedAngle'].plot()
plt.ylabel('Degrees')

plt.savefig(f"/scratch/labJackData/Accel_vs_Mount_{timestamp}.pdf")
        

In [ ]:
plt.close()